## You can download an NVIDIA model from torchhub

#### ... and then compress it as a tar.gz file, and use this for deployment:

In [1]:
!wget https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp32/versions/2/files/nvidia_ssdpyt_fp32_190826.pt

--2020-03-18 19:37:01--  https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp32/versions/2/files/nvidia_ssdpyt_fp32_190826.pt
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 52.35.21.100, 35.161.41.62
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|52.35.21.100|:443... connected.
HTTP request sent, awaiting response... 302 
Location: https://s3.us-west-2.amazonaws.com/prod-model-registry-ngc-bucket/org/nvidia/models/ssdpyt_fp32/versions/2/files/nvidia_ssdpyt_fp32_190826.pt?response-content-disposition=attachment%3B%20filename%3D%22nvidia_ssdpyt_fp32_190826.pt%22&response-content-type=application%2Foctet-stream&X-Amz-Security-Token=IQoJb3JpZ2luX2VjENv%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLXdlc3QtMiJIMEYCIQDQgkQcQ9pDRllmFXUkmQyrW6dohDEsW0WJJLAgyaxO%2BgIhAPfQV9WZeKSAzXRZwTT3q3ZjKW5GGQ%2Bk0LSSk70QiEkEKr0DCMT%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQAhoMNzg5MzYzMTM1MDI3IgxSXrpcKj5tL8heQiUqkQONTLPWf%2Bhetcq1bgG5mZ0jBrR4Hz3uN03%2BA8d8zT%2Fy9tgCLha9kXrTx1hhgqE5ZyiaoO7e9T62JWbkPCQbzq0EZLES%2F

In [2]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('nvidia_ssdpyt_fp32_190826.pt')

In [4]:
import sagemaker
sess = sagemaker.Session()

In [5]:
bucket = sess.default_bucket()

In [7]:
modelpath = sess.upload_data(
    path='model.tar.gz', bucket=bucket,
    key_prefix='sagemaker-pytorch/input')

In [8]:
modelpath

's3://sagemaker-us-east-1-497456752804/sagemaker-pytorch/input/model.tar.gz'

In [9]:
from sagemaker import get_execution_role
role = get_execution_role()

In [10]:
%%writefile transform_script.py

import torch
import os

def model_fn(model_dir):
    model = torch.load(os.path.join(model_dir, 'nvidia_ssdpyt_fp32_190826.pt')
    return model

Overwriting transform_script.py


In [12]:
from sagemaker.pytorch.model import PyTorchModel

In [13]:
pytorch_model = PyTorchModel(model_data=modelpath, role=role,
                             entry_point='transform_script.py',
                             framework_version='1.4.0')

predictor = pytorch_model.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1, wait=False)

## Also, you can download the model from torchhub with this API

In [14]:
import torch 
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)

Using cache found in /home/ec2-user/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [15]:
!ls

model.tar.gz			tmp.tar.gz
nvidia_ssdpyt_fp32_190826.pt	transform_script_hub.py
nvidia_ssdpyt_fp32_190826.pt.1	transform_script.py
Pytorch BYOM from NGC.ipynb	u.item
tmp


## Or better, download the model from torch hub on the fly

In [25]:
%%writefile transform_script_hub.py

import torch
import os

def model_fn(model_dir):
    model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math='fp32',map_location='cpu')
    return model

Overwriting transform_script_hub.py


In [26]:
#PyTorchModel requires a non-empty, model_data file
from sagemaker.pytorch.model import PyTorchModel
from sagemaker import get_execution_role
role = get_execution_role()

!echo "tmp content" > tmp
!tar -zcvf ./tmp.tar.gz tmp
pytorch_model = PyTorchModel(model_data = 'file://tmp.tar.gz',
                             role=role,
                             entry_point='./transform_script_hub.py',
                             framework_version='1.4.0')

predictor = pytorch_model.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1, endpoint_name='nvidia-ssd-pytorch-cpu')


tmp
---------------!

In [27]:
import numpy as np
import urllib

# METHOD #1: OpenCV, NumPy, and urllib
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    return image

In [28]:
predictor.predict(url_to_image('https://upload.wikimedia.org/wikipedia/commons/2/25/Postmen_Office_Room.jpg'))

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/nvidia-ssd-pytorch-cpu in account 497456752804 for more information.